### Introduction

In this notebook, we aim to evaluate the performance and consistency of our pre-trained coin classification model on the third largest class in our dataset. The model was originally trained on the two largest classes. By testing the third class, we seek to understand whether it shares similarities with the classes the model was trained on and if the model can accurately classify images from this third class.

**Key Objectives**:
1. **Load the Pre-trained Model**: We will load the model that was trained on the two largest classes to ensure consistency in our evaluation.
2. **Prepare the Third Class Dataset**: Images from the third largest class will be loaded and preprocessed to match the format used in the original training.
3. **Evaluate Model Performance**: The pre-trained model will be tested on the third class dataset to assess its performance.
4. **Analyze Consistency**: We will analyze the consistency of the model's classification results on the third class to determine if the model can generalize its learned features to this new class.

By the end of this notebook, we will gain insights into the model's ability to handle unseen classes and its potential for generalization beyond the initial training set. This analysis will help us understand the model's strengths and limitations, guiding future improvements and training strategies.

---


In [1]:
# Importing necessary libraries
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input, VGG16
from tf_keras_vis.gradcam import Gradcam
from tf_keras_vis.gradcam_plus_plus import GradcamPlusPlus
from tf_keras_vis.utils.scores import CategoricalScore
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear
from tf_keras_vis.utils import normalize
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout

import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import random

We are testing the model on `al-Mahdiyah` class

In [3]:
class_label = 'al-Mahdiyah'

data_path = r"Combined_images"

file_names = []

# Iterate over each file in the specified directory
for filename in os.listdir(data_path):
    if filename.endswith(('.jpg', '.png', '.tif', '.JPG')):
        file_names.append(filename)

# Function to extract the number from the image filename
def extract_number_from_filename(filename):
    return filename.split('_')[1].split('.')[0]

image_names = []

for name in file_names:
    image_names.append(extract_number_from_filename(name))

In [4]:
# Function to read the CSV and create a dictionary of image names to labels
def read_csv_to_dict(csv_file):
    df = pd.read_csv(csv_file)

    # Extract image names and labels
    image_names = df.iloc[:, 0].values
    labels = df.iloc[:, -1].values

    # Create a list of image-label pairs
    image_label_array = np.array([[name, label] for name, label in zip(image_names, labels)])
    
    # Convert the NumPy array to a dictionary
    image_label_dict = {row[0]: row[1] for row in image_label_array}
    return image_label_dict

In [5]:
# Main execution
csv_file_path = 'image_labels.csv'
image_label_dict = read_csv_to_dict(csv_file_path)

# Function to load and preprocess images
def load_and_preprocess_image(file_path):
    img = load_img(file_path, target_size=(224, 224))
    img_array = img_to_array(img)
    return img_array

In [8]:
# Search for image files and load images and labels
images = []
for key, label in image_label_dict.items():
    found = False
    for root, _, files in os.walk(data_path):
        for file in files:
            if key in file and label == class_label:
                file_path = os.path.join(root, file)
                img_array = load_and_preprocess_image(file_path)
                images.append(img_array)
                found = True
                break
        if found:
            break

# Convert to numpy arrays
images = np.array(images)

In [2]:
# Load the model
model = load_model('fine_tuned_vgg16.h5')

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
# Predict the images using the model
predictions = model.predict(images)

2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 563ms/step


In [17]:
predictions = np.argmax(predictions, axis=1)
print(predictions)

[1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1]


In [ ]:
# We can see that except for 3 images, all the images are classified as class 1.
# This can help us infer that maybe'al-mahdiyah' class is similar to class 1.